In [1]:
from dataclasses import dataclass


In [75]:
from typing import Generator

In [2]:
@dataclass
class Grille:
    """Grille partiellement remplie de Sudoku.
    
    0 représentera une case vide.
    """
    cases: list[int]
    
    def __post_init__(self):
        if len(self.cases) != 16:
            raise ValueError("Il doit y avoir exactement 16 cases.")
        if any(case < 0 or case > 4 for case in self.cases):
            raise ValueError("Les valeurs permises sont 0,1,2,3 ou 4!")

In [25]:
def detecte_probleme_ligne(grille: Grille) -> bool:
    """Détecte la répétition sur une ligne."""
    for i in (0, 4, 8, 12):
        for valeur in (1, 2, 3, 4):
            if grille.cases[i:i+4].count(valeur) > 1:
                return True
    return False

In [33]:
def detecte_probleme_colonne(grille: Grille) -> bool:
    """Détecte la répétition sur une colonne."""
    for i in (0, 1, 2, 3):
        for valeur in (1, 2, 3, 4):
            if grille.cases[i::4].count(valeur) > 1:
                return True
    return False

In [35]:
def detecte_probleme_bloc(grille: Grille) -> bool:
    """Détecte la répétition sur un bloc."""
    for i in (0, 2, 8, 10):
        for valeur in (1, 2, 3, 4):
            if [grille.cases[i+j] for j in (0, 1, 4, 5)].count(valeur) > 1:
                return True
    return False

In [15]:
def est_valide(grille: Grille) -> bool:
    """Détecte les répétition lignes/colonnes/sousgrilles."""
    if detecte_probleme_ligne(grille):
        return False
    if detecte_probleme_colonne(grille):
        return False
    if detecte_probleme_bloc(grille):
        return False
    return True

# Exercice

1. Codez 4 tests pour la fonction `est_valide` avec 
   - grille valide
   - répétition ligne
   - répétition colonne
   - répétition sous-grille
2. Codez la fonction

In [37]:
assert True == est_valide(
    grille=Grille(
        cases=[1, 2, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    )
)

In [38]:
assert False == est_valide(
    grille=Grille(
        cases=[1, 2, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    )
)

In [39]:
assert False == est_valide(
    grille=Grille(
        cases=[1, 2, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
    )
)

In [40]:
assert False == est_valide(
    grille=Grille(
        cases=[1, 2, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    )
)

# Exercice

1. Codez une fonction `est_remplie` détectant si il reste des cases vides
2. Adaptez la fonction `cherche_chemin` pour aller d'une grille initiale et à une grille remplie valide.

In [42]:
def est_remplie(grille: Grille) -> bool:
    """Détecte l'absence de case vide."""
    return grille.cases.count(0) == 0
        

In [43]:
assert False == est_remplie(
    grille=Grille(
        cases=[1, 2, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    )
)

In [44]:
assert True == est_remplie(
    grille=Grille(
        cases=[1, 2, 3, 4, 3, 4, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1]
    )
)

In [45]:
class PasDeSolution(Exception):
    """Représente l'absence de solution au Sudoku."""
    pass

In [76]:
def genere_voisines(grille: Grille) -> Generator[Grille, None, None]:
    """Renvoie les 4 façons de remplir la première case."""
    nouvelle_cases = [case for case in grille.cases]
    position_zero = nouvelle_cases.index(0) #attention peut planter
    
    for remplissage  in  (1, 2, 3, 4):
        nouvelle_cases[position_zero] = remplissage
        yield Grille(cases=nouvelle_cases)
                
    

In [71]:
# A modifier
assert genere_voisines(
    grille=Grille(
        cases=[0, 2, 3, 4, 3, 0, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1],
    )
) == [
    Grille(
        cases=[1, 2, 3, 4, 3, 0, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1],
    ),
    Grille(
        cases=[2, 2, 3, 4, 3, 0, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1],
    ),
    Grille(
        cases=[3, 2, 3, 4, 3, 0, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1],
    ),
    Grille(
        cases=[4, 2, 3, 4, 3, 0, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1],
    )
]

[Grille(cases=[4, 2, 3, 4, 3, 0, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1]), Grille(cases=[4, 2, 3, 4, 3, 0, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1]), Grille(cases=[4, 2, 3, 4, 3, 0, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1]), Grille(cases=[4, 2, 3, 4, 3, 0, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1])]


AssertionError: 

In [46]:
def resoud_sudoku(grille: Grille) -> Grille:
    """Resoud le sudoku correspondant à grille au départ.
    
    Genere PasDeSolution s'il le problème n'est pas soluble.
    """
    if est_remplie(grille):
        if est_valide(grille):
            return grille
        else:
            raise PasDeSolution
    else:
        for voisine in genere_voisines(grille):
            if est_valide(voisine):
                try:
                    solution = resoud_sudoku(voisine)
                except PasDeSolution:
                    pass
                else:
                    return solution
        raise PasDeSolution       

In [55]:
assert resoud_sudoku(
    grille=Grille(
        cases=[1, 2, 3, 4, 3, 4, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1],
    )
) == Grille(
        cases=[1, 2, 3, 4, 3, 4, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1],
    )

AssertionError: 

In [56]:
assert resoud_sudoku(
    grille=Grille(
        cases=[0, 2, 3, 4, 3, 0, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1],
    )
) == Grille(
        cases=[1, 2, 3, 4, 3, 4, 1, 2, 2, 1, 4, 3, 4, 3, 2, 1],
    )

AssertionError: 

# Rajouter les topos sur

1. Slicing
2. Gestion d'erreur
3. Récursion
4. Méthodes de list
5. Attention à la mutation
6. Itérateurs/Générateurs

In [77]:
x = [1, 2, 3]
print(x)
y = x
print(y)
y[0] = 100
print(y)
print(x)

[1, 2, 3]
[1, 2, 3]
[100, 2, 3]
[100, 2, 3]


In [78]:
for i in x:
    print(i)

100
2
3


In [79]:
it = iter(x)
while True:
    try:
        i = next(it)
    except StopIteration:
        break
    else:
        print(i)

100
2
3


In [80]:
def fibonacci():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

In [81]:
fib = fibonacci()

In [111]:
next(fib)

514229